# **CNN** *Pytorch*

***
### *Imports...*

In [ ]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.transforms import v2
from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchmetrics
import torchvision
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

***
### *CNN...*
**Convolutional layer**
- In-channels: 1 for greyscale, 3 for RGB. 
- Out-channels: Controls model complexity. Overfitting vs. underfitting. 
- Kernelsize: Kernelsize 3x3.
- Stride: Controls how fast the filter moves across the image. 1 = 1 pixel at a time for more detail. 
- Padding: Helps control output size. 

**Pooling layer**
- Kernelsize: Looks at 2x2 blocks.
- Stride: Moves 2 pixels per step.

In [ ]:

class CNN(nn.Module):
    def __init__(self, in_channels, num_classes=26): 
       
        super(CNN, self).__init__()

        #1ST CONVOLUTIONAL LAYER
        self.conv1 = nn.Conv2d( 
            in_channels=in_channels,  
            out_channels=32, 
            kernel_size=3, 
            stride=1,  
            padding=1) 
         
        self.bn1 = nn.BatchNorm2d(32)
        
        #POOLING LAYER
        self.pool = nn.MaxPool2d( 
            kernel_size=2, 
            stride=2)
        
        #2ND CONVOLUTIONAL LAYER
        self.conv2 = nn.Conv2d(  
            in_channels=32,  
            out_channels=64, 
            kernel_size=3, 
            stride=1, 
            padding=1)
        
        self.bn2 = nn.BatchNorm2d(64)

        #DROPOUT LAYER (FOR REGULARIZATION)
        self.dropout = nn.Dropout(p=0.3) 
        
        #FULLY CONNECTED LAYER
        self.fc1 = nn.Linear(  
            64 * 7 * 7, 
            num_classes)  

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # Apply first convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling

        x = F.relu(self.bn2(self.conv2(x)))  # Apply second convolution and ReLU activation
        x = self.pool(x)           # Apply max pooling

        x = x.view(x.size(0),-1)  # Flatten the tensor
        x = self.dropout(x)         #Reduce chance of overfitting
        x = self.fc1(x)            # Apply fully connected layer

        return x


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu" #Prefers GPU else CPU 

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    device = "cpu"
    print("Using CPU")

***
### *Hyperparamethers...*

In [ ]:
input_size = 784  # 28x28 pixels (not directly used in CNN)
num_classes = 26  # Letters A-Z
learning_rate = 0.01
batch_size = 64 #Number of imahes processed at once 
num_epochs = 30 #Number of batches to be processed 

***
### *Load data...*

In [ ]:
#Reformatting and resizing images, and saving them in the variable Transform for later use.
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  
    transforms.Resize((28,28),antialias=True), 

    transforms.RandomRotation(10),
    transforms.RandomAffine(
        degrees=0,
        translate=(0.1,0.1),
        scale=(0.9,1.1)
    ),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.1307],std=[0.3081]),
])

train_dataset = datasets.EMNIST(root='emnist-letters-train',split='letters', train=True, download=True, transform=transform, target_transform=lambda y: y - 1)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.EMNIST(root='emnist-letters-test', split='letters', train=False, download=True, transform=transform, target_transform=lambda y: y - 1)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

***
### *Initialize network with loss and optimizer...*

In [ ]:
model = CNN(in_channels=1, num_classes=num_classes).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

### Train network and evaluate

In [ ]:
train_losses, train_accuracies = [], []
test_losses, test_accuracies = [], []

for epoch in range(num_epochs):
    print(f"Epoch [{epoch + 1}/{num_epochs}]")
    print(f"Learning rate: {optimizer.param_groups[0]['lr']:.6f}")

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # TRAINING 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    model.train()
    train_correct, train_total, train_loss = 0, 0, 0.0

    for batch_index, (data, targets) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        scores = model(data) # Forward
        loss = criterion(scores, targets)

        loss.backward() # Backward
        optimizer.step()

        batch_size = targets.size(0)
        train_loss += loss.item() * batch_size
        
        _, preds = scores.max(1)
        train_correct += (preds == targets).sum().item()
        train_total += batch_size
    
    train_loss /= train_total
    train_acc = 100.0 * train_correct / train_total
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)

    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # EVALUATION 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    model.eval()
    test_correct, test_total, test_loss = 0, 0, 0.0

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            scores = model(x)
            
            loss = criterion(scores, y)
            batch_size = y.size(0)
            test_loss += loss.item() * batch_size

            _, predictions = scores.max(1) 
            test_correct += (predictions == y).sum().item() 
            test_total += batch_size

    test_loss /= test_total
    test_acc = float(test_correct) / float(test_total) * 100.0
    test_losses.append(test_loss)
    test_accuracies.append(test_acc)
        
    print(
        f"~~~~~ TRAINING: ~~~~~ \n "
        f"Training accuracy: {train_acc:.2f}. \n "
        f"Training loss: {train_loss:.4f}. \n "
        f" \n "
        f"~~~~ EVALUATING: ~~~~ \n " 
        f"Test accuracy: {test_acc:.2f}. \n "
        f"Test loss: {test_loss:.4f}. \n "
        f" \n "
        )
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # SCHEDULER 
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    scheduler.step()

    print()
    

In [ ]:
epochs = range(1, num_epochs+1)
plt.figure(figsize=(10,4))
plt.plot(epochs, train_accuracies, label="Train Accuracy")
plt.plot(epochs, test_accuracies, label="Test Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.title("Training vs Test Accuracy")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(10,4))
plt.plot(epochs, train_losses, label="Train Loss")
plt.plot(epochs, test_losses, label="Test Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Test Loss")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
def letter_confusion(loader, model):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            scores = model(x) 

            _, preds = scores.max(1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(y.cpu().numpy())

    cm = confusion_matrix(all_targets, all_preds)
    plt.figure(figsize=(12,10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

    model.train()